### Chapter 8
### Categorical Encoding with the Housing Price & Titanic Dataset
This notebook is a companion to the book XGBoost for Regression Predictive Modeling and Time Series Analysis. This chapter is a hands on guide for various categorical encoding techniques designed to give you hands-on experience with various categorical encoding techniques

### Citing some the sources 

- [kaggle Houysing price prediction](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/) - Kaggle kernel
- [Titanic dataset](https://www.openml.org/search?type=data&status=active&id=40945&sort=runs) - OpenML Datasets

Citation for the housing price dataset: `Dean De Cock(2011) Ames, Iowa: Alternative to the Boston Housing Data as an End of Semester Regression Project, Journal of Statistics Education, 19:3, DOI: 10.1080/10691898.2011.11889627.`

### In this notebook we walk through the following encoding techniques for categorical data:

- One Hot Encoding
- One Hot encoding with Frequent Categories
- Count or Frequency Encoding
- Ordered Integer Encoding
- Mean Encoding
- Probability Ration Encoding
- Weight of Evidence Encoding
- Engineering Rare Categories